In [ ]:
import pandas as pd
import numpy as np

In [ ]:
x = pd.read_excel("testa.xlsx", header=None, usecols=[0, 46, 47, 48, 49, 50, 51, 52, 53])
# x = pd.read_excel("testa.xlsx", header=None)
x

In [ ]:
import re

In [ ]:
"""正常- 18:59   缺卡(次日05:59);|	正常- 09:12; 22:00	|正常- 22:06   缺卡(次日05:59);	|正常- 20:19; 22:11	|正常- 17:41; 22:11	|正常- 22:13   缺卡(次日05:59);	|正常- 09:43; 22:22"""

In [ ]:
def find_time(text): # 从字符串中找到所有时间
    matches = re.findall(r"\b\d{2}:\d{2}\b(?![^\(]*\))", text)
    return matches


In [ ]:
personnes = x[4:]
personnes

In [ ]:
personnes.shape

In [ ]:
def time_diff(time2, time1, format=True):
    h1, m1 = time1.split(":")
    h2, m2 = time2.split(":")
    h1, m1, h2, m2 = int(h1), int(m1), int(h2), int(m2)
    if h2 < h1:
        h2 += 24
    mins = (h2 - h1) * 60 + (m2 - m1)
    if format:
        return f"{mins // 60}:{mins % 60}"
    return mins


time_diff('22:00', '09:12')

In [ ]:
class MyTime:
    def __init__(self, time_str):
        self.h, self.m = map(int, time_str.split(":"))

    def __sub__(self, other):
        if self.h < other.h:
            self.h += 24
        mins = (self.h - other.h) * 60 + (self.m - other.m)
        formatted = f"{mins // 60}:{mins % 60}"
        return MyTime(formatted)

    def __add__(self, other):
        mins = self.h * 60 + self.m + other.h * 60 + other.m
        formatted = f"{mins // 60}:{mins % 60}"
        return MyTime(formatted)

    def __str__(self):
        return f"{self.h:02d}:{self.m:02d}"

    def __repr__(self):
        return self.__str__()

    def __lt__(self, other):
        return self.h < other.h or (self.h == other.h and self.m < other.m)

    def __gt__(self, other):
        return self.h > other.h or (self.h == other.h and self.m > other.m)


time2 = MyTime("22:00")
time1 = MyTime("09:12")
time2 - time1, time1 < time2, time1 > time2, time1 - time2

In [ ]:
def calculate_time_diffs(personnes):
    data = {}
    for i in range(len(personnes)):
        items = personnes.iloc[i]
        name = items[0]
        times = []
        for item in items[1:-1]:
            ts = find_time(item)
            if len(ts) < 2:
                times.append(MyTime('00:00'))
                continue
            time2 = MyTime(ts[-1])
            time1 = MyTime(ts[0])
            times.append(time2 - time1)
        data[name] = times
    return data

data = calculate_time_diffs(personnes)

In [ ]:
def time_check_passed(times, weekdays=3, weekend=8):
    weekdays_idx = [0, 3, 4, 5, 6]
    weekend_idx = [1, 2]
    work_days = 0
    weekend_hours = MyTime("00:00")
    for idx in weekdays_idx:
        if times[idx] > MyTime("00:30"):  # 工作日打卡时间超过30算合格
            work_days += 1
    for idx in weekend_idx:
        weekend_hours = weekend_hours + times[idx]
    weekend_mins = weekend * 60
    weekend_time = MyTime(f"{weekend_mins // 60}:{weekend_mins % 60}")
    return work_days >= weekdays and weekend_hours > weekend_time  # 工作日天数、周末工作时长



In [ ]:
import openpyxl
from openpyxl.styles import PatternFill
import os

In [ ]:
def process_xlsx(file_path, x, y):
    """处理 Excel 文件并标记不符合条件的行"""
    x = pd.read_excel(file_path, header=None, usecols=[0, 46, 47, 48, 49, 50, 51, 52, 53])
    personnes = x[4:]
    data = calculate_time_diffs(personnes)
    wb = openpyxl.load_workbook(file_path)
    ws = wb.active
    for i in range(len(personnes)):
        name = personnes.iloc[i][0]
        if not time_check_passed(data[name]):
            for j in range(1, 8):
                ws.cell(i + 1+4, j).fill = PatternFill(start_color="FF0000", end_color="FF0000", fill_type="solid")
    base = os.path.basename(file_path)
    wb.save(f"output_{base}")
    print(f"处理完成，结果已保存为 output_{base}")

process_xlsx("testa.xlsx", 3, 8)
